In [2]:
#INSTALL LIBRARIES------------------------------------------
!pip install transformers scikit-learn datasets wandb

In [1]:
#IMPORTS-----------------------------
from pprint import pprint
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from torch.nn import Linear, ReLU
import pdb
import numpy as np, torch, random as rnd, torch.nn as nn, wandb
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import sys

from torch.nn.functional import cosine_similarity
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [2]:


adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

training_set = ds["train"]

validation_set = ds["validation"]

test_set = ds["test"]

pprint(adversarial[0])
pprint(training_set[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'cid': 58846,
 'hypothesis': 'The number of sitcoms from France in which Johnny Galecki has '
               'played a character is greater or equal to 2',
 'label': 'NEUTRAL',
 'part': 'manual_adversarial',
 'premise': 'Johnny Galecki . He is known for playing David Healy in the ABC '
            'sitcom Roseanne from 1992 -- 1997 and Dr. Leonard Hofstadter in '
            'the CBS sitcom The Big Bang Theory since 2007 .'}
{'hypothesis': 'Roman Atwood is a content creator.',
 'id': '150448',
 'label': 'ENTAILMENT',
 'premise': 'Roman Atwood . He is best known for his vlogs , where he posts '
            'updates about his life on a daily basis . His vlogging channel , '
            "`` RomanAtwoodVlogs '' , has a total of 3.3 billion views and "
            '11.9 million subscribers . He also has another YouTube channel '
            "called `` RomanAtwood '' , where he posts pranks .",
 'srl': {'hypothesis': {'annotations': [{'englishPropbank': {'frameName': 'be.01',
              

In [ ]:

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#model = RobertaModel.from_pretrained('roberta-base')

In [ ]:
f1 = "test per vedere come va"
f2 = "questa è una prova"

tokenized = tokenizer(f1+ tokenizer.eos_token + f2, return_tensors='pt', padding='max_length', max_length=40, return_token_type_ids=True)

print(tokenized)
print(tokenized["input_ids"].shape)
print(type(tokenized))

out = model(**tokenized)
print(out['last_hidden_state'].shape)

{'input_ids': tensor([[    0, 21959,   228,   748,   196,  2816,   283, 13205,     2, 23627,
           102,   952, 11423,   542,   102, 11570,   102,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([1, 40])
<class 'transformers.tokenization_utils_base.BatchEncoding'>
torch.Size([1, 40, 768])


In [ ]:

print(out.last_hidden_state.mean(dim=-1).squeeze().shape)

torch.Size([21])


In [4]:
class NLIDataset(Dataset):



    def __init__(self, data):

      self.encode_labels = {'CONTRADICTION': 0, 'NEUTRAL': 1, 'ENTAILMENT': 2}

      self.data = []

      for example in data:
          #breakpoint()
          inputs = tokenizer(example["premise"] + tokenizer.eos_token + example["hypothesis"], return_tensors='pt')
          token_ids = inputs['input_ids'][0]
          sep_token_id = tokenizer.sep_token_id

          # Trovare la posizione del token di separazione
          sep_index = (token_ids == sep_token_id).nonzero(as_tuple=True)[0].tolist()[0]

          element = {
              "input": inputs,
              "label": self.encode_labels[example["label"]],
              "separator_index": sep_index
              }
          self.data.append(element)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def collate(self, batch):
        x = []
        y = []
        z = []
        attention_mask = []
        #print(batch)
        #breakpoint()
        # Iterare sull'array di dizionari una sola volta
        for d in batch:
            x.append(d['input']['input_ids'].squeeze())
            y.append(d['label'])
            z.append(d['separator_index'])
            attention_mask.append(d['input']['attention_mask'].squeeze())
       #breakpoint()
        # Convertire le liste di valori in tensori PyTorch
        x = pad_sequence(x, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
        y = torch.tensor(y)
        z = torch.tensor(z)
        return x,attention_mask, y, z



    def get_dataloader(self, batch_size, pos_num):
        self.pos_num = pos_num
        return DataLoader(self, batch_size=batch_size, shuffle=True, collate_fn = self.collate)

















In [5]:
# prompt: create a torch model using roberta and a linear layer

class RobertaClassifier(nn.Module):
  def __init__(self, num_labels=3):
    super(RobertaClassifier, self).__init__()
    self.roberta = RobertaModel.from_pretrained('roberta-base')
    self.roberta_similarity = RobertaModel.from_pretrained('roberta-base')
    self.linear = nn.Linear(self.roberta.config.hidden_size+1, num_labels)

  def forward(self, input_ids, attention_mask, separators):

    breakpoint()
    with torch.no_grad():
      outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
      premise = outputs.last_hidden_state[:,0, :]
      hypotehsis = outputs.last_hidden_state[torch.arange(outputs.last_hidden_state.size(0)),separators, :]

      similarities = cosine_similarity(premise, hypotehsis, dim=-1)
      del premise, hypotehsis
      #breakpoint()

    outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
    pooled_output = outputs.pooler_output
    del outputs
    pooled_output = torch.cat((pooled_output, similarities.unsqueeze(dim=1)), dim=1)
    # outputs.last_hidden_state.mean(dim=-1)
    logits = self.linear(pooled_output)
    return logits


In [ ]:
# Function to print a progress bar
def print_progress_bar(percentuale: float, lunghezza_barra: int = 30, text: str="") -> None:
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% complete " + text)
    sys.stdout.flush()

In [ ]:


class Trainer():

    def __init__(self, model,train_dataloader, validation_dataloader, optimizer, loss_function, device, test_dataloader=None):
        self.model = model.to(device)
        self.train_dataloader = train_dataloader
        self.validation_dataloader = validation_dataloader
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.device = device
        self.test_dataloader = test_dataloader


    @staticmethod
    def evaluation_parameters(y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred).ravel()
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        return cm, precision, recall, f1, accuracy


    def train(self, epochs: int, use_wandb: bool = False, config: dict = {}, name: str="", target_f1: float=0.0, freezing=[]):
        best_model = None
        save = False
        if use_wandb:
            wandb.init(
                # Set the project where this run will be logged
                project="nlp-hw-1b",
                name=name,
                # Track hyperparameters and run metadata
                config=config
            )
        for epoch in range(epochs):
            if epoch in freezing:
                self.model.freeze(epoch)
            self.model.train()  # Set the model to training mode
            total_loss = 0
            #breakpoint()
            for i, batch in enumerate(self.train_dataloader):
                print_progress_bar(i / len(self.train_dataloader), text=f" | training epoch {epoch}")
                # Get the inputs and targets from the batch
                inputs, targets, lens = batch

                # Zero the gradients
                self.optimizer.zero_grad()
                # Forward pass
                outputs = self.model((inputs, lens))
                #print("outputs = ", outputs,"\ntargets = ", targets)
                #breakpoint()
                # Compute loss
                loss = self.loss_function(outputs, targets)
                #print(loss)
                # Backward pass and optimize
                loss.backward()
                self.optimizer.step()
                # Accumulate the total loss
                total_loss += loss.item()

            # Print the average loss for this epoch
            validation_loss, precision, recall, f1, accuracy = self.validate(use_wandb)
            if f1 > target_f1:
                best_model = self.model.state_dict()
                target_f1 = f1
                save = True
            if use_wandb:
                wandb.log({"validation_loss": validation_loss,
                      "precision": precision,
                      "recall": recall,
                      "f1": f1,
                      "accuracy": accuracy,
                      "train_loss": total_loss / len(self.train_dataloader)})
        if save:
            torch.save(best_model, name + f'-{target_f1}.pth')
        print(target_f1)
        if use_wandb:
            wandb.finish()



    def validate(self, use_wandb: bool = False, test=False):
        dataloader = self.test_dataloader if test else self.validation_dataloader
        if dataloader is None:
            print("empty dataloader!")
            exit(1)
        self.model.eval()  # Set the model to evaluation mode
        total_loss = 0
        all_predictions = torch.tensor([])
        all_targets = torch.tensor([])
        with torch.no_grad():  # Do not calculate gradients
            for i, batch in enumerate(self.validation_dataloader):
                print_progress_bar(i / len(dataloader), text=" | validation")
                # Get the inputs and targets from the batch
                inputs, targets, lens = batch

                # Forward pass
                outputs = self.model((inputs,lens))
                # Compute loss
                #breakpoint()
                loss = self.loss_function(outputs, targets)
                # Accumulate the total loss
                total_loss += loss.item()
                # Store predictions and targets
                all_predictions = torch.cat((all_predictions, outputs.squeeze().round().cpu()))
                all_targets = torch.cat((all_targets, targets.cpu()))
        validation_loss = total_loss / len(self.validation_dataloader)
        #breakpoint()
        precision = precision_score(all_targets, all_predictions)
        recall = recall_score(all_targets, all_predictions)
        f1 = f1_score(all_targets, all_predictions)
        accuracy = accuracy_score(all_targets, all_predictions)
        return validation_loss, precision, recall, f1, accuracy




In [6]:
del training_set

dataset = NLIDataset(validation_set)


dataloader = dataset.get_dataloader(batch_size=32, pos_num=1)

model = RobertaClassifier()

batch = next(iter(dataloader))

model(batch[0], batch[1], batch[3])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispa

> <ipython-input-5-fac945463f68>(13)forward()
     11 
     12     breakpoint()
---> 13     with torch.no_grad():
     14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
     15       premise = outputs.last_hidden_state[:,0, :]

ipdb> n
> <ipython-input-5-fac945463f68>(14)forward()
     12     breakpoint()
     13     with torch.no_grad():
---> 14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
     15       premise = outputs.last_hidden_state[:,0, :]
     16       hypotehsis = outputs.last_hidden_state[torch.arange(outputs.last_hidden_state.size(0)),separators, :]

ipdb> n
> <ipython-input-5-fac945463f68>(15)forward()
     13     with torch.no_grad():
     14       outputs = self.roberta_similarity(input_ids=input_ids, attention_mask=attention_mask)
---> 15       premise = outputs.last_hidden_state[:,0, :]
     16       hypotehsis = outputs.last_hidden_state[torch.arange(outputs.last_hidden_state.s


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 361, in set_quit
    sys.settrace(None)

